In [102]:
#import statements
from pyspark.sql import SparkSession
from pyspark.sql import HiveContext
hive_context = HiveContext(sc)

In [103]:
import folium
import html

In [104]:
spark = SparkSession.builder.enableHiveSupport().appName('ReadWriteData').getOrCreate()
sc = spark.sparkContext

In [105]:
#check to see if thata is available in HDFS
!hdfs dfs -ls /user/zxie8/big_data/

Java HotSpot(TM) 64-Bit Server VM warning: ignoring option MaxPermSize=512M; support was removed in 8.0
Found 15 items
drwxr-xr-x   - zxie8 zxie8          0 2020-08-20 16:02 /user/zxie8/big_data/CF_Model_1
drwxr-xr-x   - zxie8 zxie8          0 2020-08-25 02:06 /user/zxie8/big_data/CF_Model_VG_1
-rw-r--r--   3 zxie8 zxie8       4215 2020-08-19 18:49 /user/zxie8/big_data/Movie_Collect.csv
drwxr-xr-x   - zxie8 zxie8          0 2020-08-15 03:05 /user/zxie8/big_data/airports
drwxr-xr-x   - zxie8 zxie8          0 2020-08-20 16:03 /user/zxie8/big_data/als
drwxr-xr-x   - zxie8 zxie8          0 2020-08-20 18:10 /user/zxie8/big_data/als_1
drwxr-xr-x   - zxie8 zxie8          0 2020-08-25 13:23 /user/zxie8/big_data/als_VG_1
drwxr-xr-x   - zxie8 zxie8          0 2020-08-26 02:12 /user/zxie8/big_data/als_VG_2
drwxr-xr-x   - zxie8 zxie8          0 2020-08-26 02:32 /user/zxie8/big_data/als_VG_3
drwxr-xr-x   - zxie8 zxie8          0 2020-08-08 05:32 /user/zxie8/big_data/instacart
drwxr-xr-x   - zxie8 z

### Reading Data

In [106]:
buz = hive_context.sql('select * from big_data_group_2.business_julian where city == "Las Vegas"')
# df=df.filter(df.id != "id")
buz = buz.withColumnRenamed("stars","buz_stars")
buz.show(5)

+--------------------+--------------------+--------------------+---------+-----+-----------+-------------+---------------+---------+------------+-------+----------+----------+-----+
|         business_id|                name|             address|     city|state|postal_code|     latitude|      longitude|buz_stars|review_count|is_open|attributes|categories|hours|
+--------------------+--------------------+--------------------+---------+-----+-----------+-------------+---------------+---------+------------+-------+----------+----------+-----+
|oiAlXZPIFm2nBCt0D...|Green World Cleaners|6870 S Rainbow Bl...|Las Vegas|   NV|      89118|   36.0639767|    -115.241463|      3.5|          81|      1|         {|      null| null|
|uZuh51lXu7tsrC8RA...|      Manolo Blahnik| 3131 Las Vegas Blvd|Las Vegas|   NV|      89109|36.1257322229|-115.1676084101|      4.5|          32|      0|         {|      null| null|
|fnZrZlqW1Z8iWgTVD...|           Carl's Jr|9595 W Tropicana Ave|Las Vegas|   NV|      8914

In [107]:
buz.count()

31631

In [108]:
# Filter
buz=buz.select('business_id', 'name','categories', 'latitude','longitude','review_count','buz_stars')

In [109]:
# df = hive_context.table("big_data_group_2.review")
rv = hive_context.sql('select * from big_data_group_2.review')
# df=df.filter(df.id != "id")
rv.show(5)

+--------------------+--------------------+--------------------+-----+-------------------+--------------------+------+-----+----+
|           review_id|             user_id|         business_id|stars|               date|                text|useful|funny|cool|
+--------------------+--------------------+--------------------+-----+-------------------+--------------------+------+-----+----+
|xQY8N_XvtGbearJ5X...|OwjRMXRC0KyPrIlcj...|-MhfebM0QIsKt87iD...|  2.0|2015-04-15 05:21:16|As someone who ha...|     5|    0|   0|
|UmFMZ8PyXZTY2Qcwz...|nIJD_7ZXHq-FX8byP...|lbrU8StCq3yDfr-QM...|  1.0|2013-12-07 03:16:52|I am actually hor...|     1|    1|   0|
|LG2ZaYiOgpr2DK_90...|V34qejxNsCbcgD8C0...|HQl28KMwrEKHqhFrr...|  5.0|2015-12-05 03:18:11|I love Deagan's. ...|     1|    0|   0|
|i6g_oA9Yf9Y31qt0w...|ofKDkJKXSKZXu5xJN...|5JxlZaqCnk1MnbgRi...|  1.0|2011-05-27 05:30:52|Dismal, lukewarm,...|     0|    0|   0|
|6TdNDKywdbjoTkize...|UgMW8bLE0QMJDCkQ1...|IS4cv902ykd8wj1TR...|  4.0|2017-01-14 21:56:57|

In [110]:
rv.count()

80211220

In [111]:
rv.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: integer (nullable = true)
 |-- funny: integer (nullable = true)
 |-- cool: integer (nullable = true)



In [112]:
# 'date' to date format
from pyspark.sql.functions import to_date

rv = rv.withColumn('date', rv['date'].substr(1, 10))
rv = rv.withColumn('date',to_date(rv.date, 'yyyy-mm-dd'))

In [113]:
rv.show(5)

+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
|           review_id|             user_id|         business_id|stars|      date|                text|useful|funny|cool|
+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
|xQY8N_XvtGbearJ5X...|OwjRMXRC0KyPrIlcj...|-MhfebM0QIsKt87iD...|  2.0|2015-01-15|As someone who ha...|     5|    0|   0|
|UmFMZ8PyXZTY2Qcwz...|nIJD_7ZXHq-FX8byP...|lbrU8StCq3yDfr-QM...|  1.0|2013-01-07|I am actually hor...|     1|    1|   0|
|LG2ZaYiOgpr2DK_90...|V34qejxNsCbcgD8C0...|HQl28KMwrEKHqhFrr...|  5.0|2015-01-05|I love Deagan's. ...|     1|    0|   0|
|i6g_oA9Yf9Y31qt0w...|ofKDkJKXSKZXu5xJN...|5JxlZaqCnk1MnbgRi...|  1.0|2011-01-27|Dismal, lukewarm,...|     0|    0|   0|
|6TdNDKywdbjoTkize...|UgMW8bLE0QMJDCkQ1...|IS4cv902ykd8wj1TR...|  4.0|2017-01-14|Oh happy day, fin...|     0|    0|   0|
+--------------------+----------

In [114]:
rv.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- date: date (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: integer (nullable = true)
 |-- funny: integer (nullable = true)
 |-- cool: integer (nullable = true)



In [115]:
#Filter
rv=rv.select('review_id','user_id', 'business_id', 'stars')

### Combining Data

In [116]:
df = buz.join(rv, on=['business_id'], how='inner')
df.show(5)

+--------------------+--------------------+----------+---------+----------+------------+---------+--------------------+--------------------+-----+
|         business_id|                name|categories| latitude| longitude|review_count|buz_stars|           review_id|             user_id|stars|
+--------------------+--------------------+----------+---------+----------+------------+---------+--------------------+--------------------+-----+
|--9e1ONYQuAa-CB_R...|Delmonico Steakhouse|      null|36.123183|-115.16919|        1759|      4.0|-ogKPaiHquD1m6Sxd...|qOODN0RsLbNQaLPN3...|  4.0|
|--9e1ONYQuAa-CB_R...|Delmonico Steakhouse|      null|36.123183|-115.16919|        1759|      4.0|yJR3BJZSeGJItuXV7...|WU3XkIdG8qZwFW2wA...|  5.0|
|--9e1ONYQuAa-CB_R...|Delmonico Steakhouse|      null|36.123183|-115.16919|        1759|      4.0|vlLBaxWA2-SHjTnMA...|KGcyC9KXloxW_6YMG...|  4.0|
|--9e1ONYQuAa-CB_R...|Delmonico Steakhouse|      null|36.123183|-115.16919|        1759|      4.0|gYa9xXB4qhczLZjm_...

In [117]:
df.count()

24448010

### Preparing data for modeling

In [118]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyspark.ml import Pipeline
from pyspark.sql.functions import *
from pyspark.sql.types import *


from pyspark.sql import Row

from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.evaluation import RegressionEvaluator


In [119]:
# Show the minimum number of reviews 
df.groupBy('user_id').count().sort("count").orderBy(["count"], descending=[0, 0]).show()

+--------------------+-----+
|             user_id|count|
+--------------------+-----+
|3lWVKWVA0DYEMIMJw...|   10|
|ob6uKIIu7cRALM0fL...|   10|
|foHqSyru1BDNuzdIs...|   10|
|5wGOTD8ZqJeVWbwmn...|   10|
|l7NCzf4eQkQ_Du-Gs...|   10|
|PcuxUs5wP3g86jQ0d...|   10|
|foCZOvOpMpn36MK4K...|   10|
|xagGcvcPF-fxsyCgO...|   10|
|8npuzCxM6rMMqvlyP...|   10|
|X9rf45zpoNImG4fuX...|   10|
|vRSetBkuRMrjZvpW7...|   10|
|gv5lh2-NRqbPXFB9a...|   10|
|U5RoLDX6wIwjp4EOV...|   10|
|sowJXiMAF8mVUh0-9...|   10|
|NiOq_r84QNayFfaHL...|   10|
|FXAfppYvsI4v2tIur...|   10|
|sNnVTS6bhbzarwnsK...|   10|
|ZVx8UwKxAa4AbcmJe...|   10|
|dk7HEI-bJkAfoJH8C...|   10|
|OPE1WaiP29VEQJUNa...|   10|
+--------------------+-----+
only showing top 20 rows



In [120]:
# Each user has at least 10 reviews, so consider all of them as active users
df.select("stars").count()

24448010

In [121]:
#Unique users
df.select("user_id").distinct().count()
# 1_968_703

748960

In [122]:
#Unique business
df.select("business_id").distinct().count()

31631

In [123]:
#building df2 for training
# df2=df.select('user_id', 'business_id', 'stars')
df2=df
# df2.show(5)

In [124]:
user_newid_df=df2.select('user_id').distinct()
user_newid_df = sqlContext.createDataFrame(user_newid_df.rdd.map(lambda x: x[0]).zipWithIndex(), \
        StructType([StructField("user_id", StringType(), True),StructField("userID", IntegerType(), True)]))

user_newid_df.show(2)

+--------------------+------+
|             user_id|userID|
+--------------------+------+
|3BsiFEeiP7y_IUE1i...|     0|
|ECflfQ6kMX8DiScCF...|     1|
+--------------------+------+
only showing top 2 rows



In [125]:
user_newid_df.count()

748960

In [126]:
buz_newid_df=df2.select('business_id').distinct()
buz_newid_df = sqlContext.createDataFrame(buz_newid_df.rdd.map(lambda x: x[0]).zipWithIndex(), \
        StructType([StructField("business_id", StringType(), True),StructField("businessID", IntegerType(), True)]))

buz_newid_df.show(2)

+--------------------+----------+
|         business_id|businessID|
+--------------------+----------+
|--9e1ONYQuAa-CB_R...|         0|
|-ElDqujEn1u64ynQr...|         1|
+--------------------+----------+
only showing top 2 rows



In [127]:
buz_newid_df.count()

31631

In [128]:
# add the new businessId column the business dataframe

a = buz.alias("a")
b = buz_newid_df.alias("b")
    
business_new_df = a.join(b, col("a.business_id") == col("b.business_id"), 'inner') \
             .select([col('a.'+xx) for xx in a.columns] + [col('b.businessID')])

business_new_df.select('businessID','business_id', 'name').show(2)

+----------+--------------------+--------------------+
|businessID|         business_id|                name|
+----------+--------------------+--------------------+
|         0|--9e1ONYQuAa-CB_R...|Delmonico Steakhouse|
|         1|-ElDqujEn1u64ynQr...| McFarling Law Group|
+----------+--------------------+--------------------+
only showing top 2 rows



In [129]:
df2=df2.join(user_newid_df, ["user_id"])
df2=df2.join(buz_newid_df, ["business_id"])
df2.show(5)

+--------------------+--------------------+--------------------+----------+---------+----------+------------+---------+--------------------+-----+------+----------+
|         business_id|             user_id|                name|categories| latitude| longitude|review_count|buz_stars|           review_id|stars|userID|businessID|
+--------------------+--------------------+--------------------+----------+---------+----------+------------+---------+--------------------+-----+------+----------+
|--9e1ONYQuAa-CB_R...|3Qv2rjPXC2eNKyV5l...|Delmonico Steakhouse|      null|36.123183|-115.16919|        1759|      4.0|ho7ZJ6cCt9tCwhjQH...|  5.0|614458|         0|
|--9e1ONYQuAa-CB_R...|3Qv2rjPXC2eNKyV5l...|Delmonico Steakhouse|      null|36.123183|-115.16919|        1759|      4.0|ho7ZJ6cCt9tCwhjQH...|  5.0|614458|         0|
|--9e1ONYQuAa-CB_R...|3Qv2rjPXC2eNKyV5l...|Delmonico Steakhouse|      null|36.123183|-115.16919|        1759|      4.0|ho7ZJ6cCt9tCwhjQH...|  5.0|614458|         0|
|--9e1ONYQ

In [130]:
#Check
df2.filter(df2.businessID == 100).show(100)

+--------------------+--------------------+--------------------+----------+----------+----------+------------+---------+--------------------+-----+------+----------+
|         business_id|             user_id|                name|categories|  latitude| longitude|review_count|buz_stars|           review_id|stars|userID|businessID|
+--------------------+--------------------+--------------------+----------+----------+----------+------------+---------+--------------------+-----+------+----------+
|dk1MV0MP32Xq-iBxz...|5OPc0s5BiXbAv1pKg...|Tropical Smoothie...|      null|36.1141249|-115.30536|         286|      2.0|Em_SHmgb5d6UK7UfB...|  1.0| 40805|       100|
|dk1MV0MP32Xq-iBxz...|5OPc0s5BiXbAv1pKg...|Tropical Smoothie...|      null|36.1141249|-115.30536|         286|      2.0|Em_SHmgb5d6UK7UfB...|  1.0| 40805|       100|
|dk1MV0MP32Xq-iBxz...|5OPc0s5BiXbAv1pKg...|Tropical Smoothie...|      null|36.1141249|-115.30536|         286|      2.0|Em_SHmgb5d6UK7UfB...|  1.0| 40805|       100|
|dk1

In [131]:
df3=df2.select('userID', 'businessID', df2.stars.cast('float').alias('rating'))

In [132]:
# df3 = df3.withColumn("userID",col("userID").cast(DecimalType(38, 0))).withColumn("businessID",col("businessID").cast(DecimalType(38, 0)))
# cast(DecimalType(38, 0))

In [133]:
df3.show(5)

+------+----------+------+
|userID|businessID|rating|
+------+----------+------+
|345385|         0|   2.0|
|345385|         0|   2.0|
|345385|         0|   2.0|
|345385|         0|   2.0|
|345385|         0|   2.0|
+------+----------+------+
only showing top 5 rows



In [134]:
df3.printSchema()

root
 |-- userID: integer (nullable = true)
 |-- businessID: integer (nullable = true)
 |-- rating: float (nullable = true)



### Building CF Models

In [135]:
#Split
(train, test) = df3.randomSplit([0.8, 0.2], seed=666)

Root-mean-square error = 2.623877828241486

In [136]:
model_path = '/user/zxie8/big_data/'

Model 1

Model 2

Model 3

Root-mean-square error = 2.1197205019461247

In [151]:
alsn_model = ALSModel.load(model_path + 'als_VG_1')
# alsn_model = ALSModel.load(model_path + 'als_VG_2')
# alsn_model = ALSModel.load(model_path + 'als_VG_3')

In [152]:
# generate top 10 business recommendations for each user

userRecoms = alsn_model.recommendForAllUsers(10)

In [153]:
# add the column user_id, cache the recommendaton dataframe and show recommedations sample

a = userRecoms.alias("a")
b = user_newid_df.alias("b")
    
all_userRecoms = a.join(b, col("a.userId") == col("b.userId"), 'inner') \
             .select([col('a.'+xx) for xx in a.columns] + [col('b.user_id')])

all_userRecoms.cache()   
all_userRecoms.show(1, truncate=False)

+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------+
|userID|recommendations                                                                                                                                                                               |user_id               |
+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------+
|148   |[[27399, 4.358519], [2308, 4.351308], [61, 4.3355346], [25269, 4.3316636], [18439, 4.3287992], [28135, 4.3093123], [12470, 4.309261], [13222, 4.3076158], [22603, 4.304784], [8777, 4.294553]]|dTQQA8_CG1UDNiCDVxkqVw|
+------+----------------------------------------------------------------------------------------------------

In [154]:
# show recommendations for some user id

u_id = 'beD62otPliS9ewkWhBf6gw'

userFlatRec =  sqlContext.createDataFrame(all_userRecoms.filter(col('user_id') == u_id).rdd.flatMap(lambda p: p[1]))
userFlatRec.show()

+----------+-----------------+
|businessID|           rating|
+----------+-----------------+
|     12470|5.383881092071533|
|     14118|5.335316181182861|
|     27399|5.324843406677246|
|        61|5.318561553955078|
|     25269|5.312559604644775|
|         2|5.312485694885254|
|     29716|5.293753623962402|
|      2308| 5.28790283203125|
|      1809|5.282406330108643|
|     28135|5.277737140655518|
+----------+-----------------+



In [155]:
def showRecom(df):
    
    mp = folium.Map(location=[36.114647, -115.172813], zoom_start=12)

    for i, r in df.toPandas().iterrows():
        folium.Marker(
                    location =[r.latitude, r.longitude], 
                    popup = html.escape(r["name"]) + '<br>' + 'Stars: ' + str(r.buz_stars) + '<br>' + 'Reviews: ' + str(r.review_count),    
                    icon = folium.Icon(color='blue')).add_to(mp)

    return mp

In [156]:
def getRecom(u_id):
    

    userFlatRec =  sqlContext.createDataFrame(all_userRecoms.filter(col('user_id') == u_id).rdd.flatMap(lambda p: p[1]))

    a = userFlatRec.alias("a")
    b = business_new_df.alias("b")
    
    return a.join(b, col("a.businessId") == col("b.businessId"), 'inner') \
             .select([col('b.business_id'), col('a.rating'), col('b.name'),col('b.categories'),
                                                           col('b.buz_stars'),col('b.review_count'),
                                                           col('b.latitude'),col('b.longitude')]) \
             .orderBy("rating","buz_stars","review_count", ascending = False)

In [163]:
u_id = 'FNRQXCnpfb0ksDEZ4lANyg'

getRecom(u_id).toPandas()

,business_id,rating,name,categories,buz_stars,review_count,latitude,longitude
0,abkgaz0_23frFyJrdRZC_Q,5.054856,Nanette Lepore,None,5.0,3,36.1190372,-115.176005
1,2xSKmHrNWKFfpB9h6p3XkQ,4.968528,Nevada Legal Services,"Professional Services, Tenant and Eviction Law...",4.5,7,36.1632812,-115.1433832
2,OcUVZniPo7CnIG-Tv83XcQ,4.903667,"Karl M Larsen, OD",None,5.0,3,36.2189115,-115.2838436
3,cV8xXz0lqiUfVna710Iaag,4.890685,Sprint Store by Experts Choice,None,5.0,3,36.0855773,-115.101543
4,h1SWjGh9ewGN2Jv4RqJNDw,4.858448,Ali Lopez - Wardley Real Estate,None,5.0,3,36.180082,-115.243102
5,sN3jqoj6oG10zqwjjGY0Dw,4.846666,GlowRun,None,5.0,4,36.167053,-115.142227
6,Ck2We4s5U3DWULL8j_PRVw,4.845334,Ryoku Judo Club,None,5.0,3,36.124915,-115.191197
7,jE1yx2U1BDMddvDXsiQHvw,4.817109,Spyder,None,5.0,5,36.1629138,-115.1593311
8,YOXKs1t4o4XxMpckotRXXA,4.807481,Original Penguin,None,5.0,14,36.1092599,-115.1705243
9,XiVHxG2nUSp_bn4dBbs4Cg,4.804790,Sushi House Manda,None,5.0,6,36.102015,-115.2965319


In [164]:
u_id = 'FNRQXCnpfb0ksDEZ4lANyg'
showRecom(getRecom(u_id))

In [150]:
! pwd

/home/zxie8/code
